# Transformação do dataset

Neste _notebook_ é realizada a trasnformação do _dataset_ EEG de `.txt` para um arquivo `.npy` (_numpy array_).

In [1]:
import os
import numpy as np

In [4]:
# TO DO: Automatizar a obtenção dos dados do dataset

directory = '../dataset-s6/'

# Get list of files in directory
files = os.listdir(directory)

# Filter files that are txt and start with 'OpenBCI...'
dataset_files = [file for file in files if file.endswith('.txt') and file.startswith('OpenBCI')]

# Load data from each file into an array
data_array = []
for file in dataset_files:
    file_path = os.path.join(directory, file)
    data = np.loadtxt(file_path, dtype=float, delimiter=',', skiprows=5, usecols=range(1, 9))
    data_array.append(data)

# Tarefa 1: Cortar os dados ruidosos de cada file (posição) do data_array

# ---------------------- TO DO: excluir
# filenames de exemplo
file1 = '../dataset-s6/OpenBCI-RAW-2023-10-10_14-22-28.txt'
file2 = '../dataset-s6/OpenBCI-RAW-2023-10-25_17-43-36.txt'
file3 = '../dataset-s6/OpenBCI-RAW-2023-11-14_17-25-16.txt'

# skiprows=5 pula 5 linhas | 8 eletrodos, então 8 colunas de dados usecols 1-8
data1 = np.loadtxt(file1, dtype=float, delimiter=',', skiprows=5, usecols=range(1, 9))
data2 = np.loadtxt(file2, dtype=float, delimiter=',', skiprows=5, usecols=range(1, 9))
data3 = np.loadtxt(file3, dtype=float, delimiter=',', skiprows=5, usecols=range(1, 9))
# ----------------------

# Tarefa 2: Realizar os cortes de dados ruidosos
# data1: 500 pois são 2 segundos iniciais jogados fora (2x250)
# data2: o início foi cortado, bem como o fim (por isso os -1750)
# data3: corte dos 2 segundos iniciais também

# concatenar os vários arquivos .txts analisados em um único .npy array
data_final = np.concatenate((data1[500:,:], data2[500:-1750,:], data3[500:,:]), axis=0)

np.save('../dataset-s6/data.npy', data_final)

[[-126218.33398561 -126043.45393699 -110719.61202855 ...  -88472.92077219
  -134513.66985007   50744.94191944]
 [-126312.74775419 -126165.36035125 -110842.12193991 ...  -88588.63575323
  -134644.85223828   50636.84888325]
 [-126265.65262862 -126098.03689695 -110748.62459286 ...  -88518.27246169
  -134581.84267066   50720.93614589]
 ...
 [-126817.22662654 -126105.99411797 -110607.13805045 ...  -88017.30281321
  -134783.07542599   49024.5728522 ]
 [-126756.1169572  -126082.54713804 -110599.27023641 ...  -87981.74118778
  -134691.94736385   49057.2511026 ]
 [-126846.86503969 -126213.81893323 -110774.10558153 ...  -88127.02752674
  -134776.66047533   48915.69750496]]
